In [1]:
# MLFlow Notebook
import mlflow
from mlflow.tracking import MlflowClient
import azureml
from azureml.core import Workspace, Run, Experiment

import os, shutil

from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
import datetime as dt
from pyspark.ml.feature import OneHotEncoder, VectorAssembler

In [2]:
# set aml workspace parameters here. 
subscription_id = ""
resource_group = ""
workspace_name = ""
workspace_region = ""

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

In [3]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [4]:
# create experiment
experiment_name = 'bikeSharingDemandMLFlowAML'
exp = Experiment(workspace=ws, name=experiment_name)

In [5]:
mlflow.set_experiment(experiment_name)

run = mlflow.start_run()
run_id = run.info.run_uuid
exp_id = run.info.experiment_id
artifact_location = run.info.artifact_uri

--------------------------------------------------------------------------- 
 Exception Traceback (most recent call last)
 <command-1039153375035168> in <module> () 
 1 mlflow . set_experiment ( experiment_name ) 
 2 
 ----> 3 run = mlflow . start_run ( ) 
 4 run_id = run . info . run_uuid 
 5 exp_id = run . info . experiment_id 

 /databricks/python/lib/python3.5/site-packages/mlflow/tracking/fluent.py in start_run (run_id, experiment_id, run_name, nested) 
 109 raise Exception(("Run with UUID {} is already active. To start a nested " +
 110 "run, call start_run with nested=True" ) . format ( 
 --> 111 _active_run_stack[0].info.run_id))
 112 if run_id : 
 113 existing_run_id = run_id 

 Exception : Run with UUID 4ab8632a-21bf-4026-9a2f-322d091b067e is already active. To start a nested run, call start_run with nested=True

In [6]:
azRun = Run(exp, run_id)

In [7]:
df = (spark
      .read
      .format("csv")
      .option("inferSchema", "True")
      .option("header", "True")
      .load("/databricks-datasets/bikeSharing/data-001/day.csv")
     )
# split data
train_df, test_df = df.randomSplit([0.7, 0.3])

# One Hot Encoding
mnth_encoder = OneHotEncoder(inputCol="mnth", outputCol="encoded_mnth")
weekday_encoder = OneHotEncoder(inputCol="weekday", outputCol="encoded_weekday")

# set the training variables we want to use
train_cols = ['encoded_mnth', 'encoded_weekday', 'temp', 'hum']

# convert cols to a single features col
assembler = VectorAssembler(inputCols=train_cols, outputCol="features")

# Set linear regression model
lr = LinearRegression(featuresCol="features", labelCol="cnt")

# Create pipeline
pipeline = Pipeline(stages=[
    mnth_encoder,
    weekday_encoder,
    assembler,
    lr
])

# fit pipeline
lrPipelineModel = pipeline.fit(train_df)

# write test predictions to datetime and lastest folder
predictions = lrPipelineModel.transform(test_df)
# mlflow log evaluations
evaluator = RegressionEvaluator(labelCol = "cnt", predictionCol = "prediction")

mlflow.log_metric("mae", evaluator.evaluate(predictions, {evaluator.metricName: "mae"}))
mlflow.log_metric("rmse", evaluator.evaluate(predictions, {evaluator.metricName: "rmse"}))
mlflow.log_metric("r2", evaluator.evaluate(predictions, {evaluator.metricName: "r2"}))

In [8]:
model_nm = "bikeshare.mml"
model_output = '/mnt/azml/outputs/'+model_nm
model_dbfs = "/dbfs"+model_output
lrPipelineModel.write().overwrite().save(model_output)

In [9]:
model_name, model_ext = model_dbfs.split(".")

In [10]:
model_zip = model_name + ".zip"
shutil.make_archive(model_name, 'zip', model_dbfs)
azRun.upload_file("outputs/" + model_nm, model_zip)

Out[ 39 ]: <azureml._restclient.models.batch_artifact_content_information_dto.BatchArtifactContentInformationDto at 0x7f0708dcedd8>

In [11]:
azRun.register_model(model_name = 'model_nm', model_path = "outputs/" + model_nm)

Out[ 40 ]: Model(workspace=Workspace.create(name='demoworkspace', subscription_id='4ad547a3-bf5b-454a-925c-56e663a6d6e4', resource_group='AML'), name=model_nm, id=model_nm:2, version=2, tags={}, properties={})

In [12]:
# now delete the serialized model from local folder since it is already uploaded to run history 
shutil.rmtree(model_dbfs)
os.remove(model_zip)

In [13]:
mlflow.end_run()